---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [0]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    data = []
    State = None
    State_RegionNames = []
    with open('university_towns.txt') as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]':
                State = thisLine[:-6]
                continue
            if '(' in line:
                RegionName = thisLine[:thisLine.index('(')-1]
                State_RegionNames.append([State,RegionName])
            else:
                RegionName = thisLine
                State_RegionNames.append([State,RegionName])
            data.append(thisLine)
    df = pd.DataFrame(State_RegionNames,columns = ['State','RegionName'])
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [5]:
!pip install xlrd

    100% |████████████████████████████████| 112kB 2.6MB/s 


In [6]:
import pandas as pd
def get_recession_start():
    quarter = pd.read_excel('gdplev.xls', skiprows=219)
    quarter = quarter[['1999q4', 9926.1]]
    quarter.columns = ['Quarter','GDP']
    for i in range(2, len(quarter)):
        if (quarter.iloc[i-2][1] - quarter.iloc[i-1][1])>0 and (quarter.iloc[i-1][1] - quarter.iloc[i][1])>0:
            return quarter.iloc[i-2][0]

get_recession_start()


'2008q3'

In [7]:
def get_recession_end():
    quarter = pd.read_excel('gdplev.xls', skiprows=219)
    quarter = quarter[['1999q4', 9926.1]]
    quarter.columns = ['Quarter','GDP']
    recession_start = get_recession_start()
    recession_start_index = quarter[quarter['Quarter'] == recession_start].index.tolist()[0]
    quarter=quarter.iloc[recession_start_index:]

    for i in range(2, len(quarter)):
        if (quarter.iloc[i-2][1] - quarter.iloc[i-1][1])<0 and (quarter.iloc[i-1][1] - quarter.iloc[i][1])<0:
            return quarter.iloc[i][0]
get_recession_end()

'2009q4'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    quarter = pd.read_excel('gdplev.xls', skiprows=219)
    quarter = quarter[['1999q4', 9926.1]]
    quarter.columns = ['Quarter','GDP'] 
    recession_start = get_recession_start()
    recession_start_index = quarter[quarter['Quarter'] == recession_start].index.tolist()[0]
    recession_end = get_recession_end()
    recession_end_index = quarter[quarter['Quarter'] == recession_end].index.tolist()[0]
    quarter=quarter.iloc[recession_start_index:recession_end_index+1]
    recession_bottom = quarter['GDP'].min()
    recession_bottom_index = quarter[quarter['GDP'] == recession_bottom].index.tolist()[0]-recession_start_index
    return quarter.iloc[recession_bottom_index]['Quarter']
get_recession_bottom()

'2009q2'

In [9]:

def new_col_names():
    years = list(range(2000,2017))
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    return quar_years[:67]
new_col_names()


['2000q1',
 '2000q2',
 '2000q3',
 '2000q4',
 '2001q1',
 '2001q2',
 '2001q3',
 '2001q4',
 '2002q1',
 '2002q2',
 '2002q3',
 '2002q4',
 '2003q1',
 '2003q2',
 '2003q3',
 '2003q4',
 '2004q1',
 '2004q2',
 '2004q3',
 '2004q4',
 '2005q1',
 '2005q2',
 '2005q3',
 '2005q4',
 '2006q1',
 '2006q2',
 '2006q3',
 '2006q4',
 '2007q1',
 '2007q2',
 '2007q3',
 '2007q4',
 '2008q1',
 '2008q2',
 '2008q3',
 '2008q4',
 '2009q1',
 '2009q2',
 '2009q3',
 '2009q4',
 '2010q1',
 '2010q2',
 '2010q3',
 '2010q4',
 '2011q1',
 '2011q2',
 '2011q3',
 '2011q4',
 '2012q1',
 '2012q2',
 '2012q3',
 '2012q4',
 '2013q1',
 '2013q2',
 '2013q3',
 '2013q4',
 '2014q1',
 '2014q2',
 '2014q3',
 '2014q4',
 '2015q1',
 '2015q2',
 '2015q3',
 '2015q4',
 '2016q1',
 '2016q2',
 '2016q3']

In [11]:
def convert_housing_data_to_quarters():
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    data.drop(columns=['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=True)
    data['State'] = data['State'].map(states)
    data.set_index(['State','RegionName'],inplace=True)
    col = list(data.columns)
    col = col[0:45]
    data.drop(col,axis=1,inplace=True)

    #qs is the quarters of the year
    qs = [list(data.columns)[x:x+3] for x in range(0, len(list(data.columns)), 3)]
    
    # new columns
    column_names = new_col_names()
    for col,q in zip(column_names,qs):
        data[col] = data[q].mean(axis=1)
        
    data = data[column_names]
    return data
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [23]:
  data = convert_housing_data_to_quarters().copy()
  data = data.loc[:,'2008q3':'2009q2']
  data = data.reset_index()
  data

,State,RegionName,2008q3,2008q4,2009q1,2009q2
0,New York,New York,4.997667e+05,4.879333e+05,4.777333e+05,4.658333e+05
1,California,Los Angeles,4.695000e+05,4.439667e+05,4.262667e+05,4.139000e+05
2,Illinois,Chicago,2.320000e+05,2.270333e+05,2.237667e+05,2.197000e+05
3,Pennsylvania,Philadelphia,1.169333e+05,1.158667e+05,1.162000e+05,1.161667e+05
4,Arizona,Phoenix,1.937667e+05,1.833333e+05,1.775667e+05,1.682333e+05
5,Nevada,Las Vegas,2.133667e+05,1.949333e+05,1.812000e+05,1.643333e+05
6,California,San Diego,4.246667e+05,4.076333e+05,3.957000e+05,3.895000e+05
7,Texas,Dallas,1.121667e+05,1.099000e+05,1.076667e+05,1.051000e+05
8,California,San Jose,5.831333e+05,5.601000e+05,5.481667e+05,5.303000e+05
9,Florida,Jacksonville,1.547333e+05,1.491667e+05,1.452667e+05,1.408333e+05


In [12]:
def run_ttest():
  data = convert_housing_data_to_quarters().copy()
  data = data.loc[:,'2008q3':'2009q2']
  data = data.reset_index()
  def price_ratio(row):
    return (row['2008q3'] - row['2009q2'])/row['2008q3']

  data['up&down'] = data.apply(price_ratio,axis=1) 
  uni_town = get_list_of_university_towns()['RegionName']
  uni_town = set(uni_town)
  def check_uni_town(row):
    if(row['RegionName'] in uni_town):
      return 1
    else:
      return 0
  data['is_uni']=data.apply(check_uni_town,axis=1)
  
  not_uni=data[data['is_uni']==0].loc[:,'up&down'].dropna().mean()
  uni=data[data['is_uni']==1].loc[:,'up&down'].dropna().mean()
  p_val = list(ttest_ind(not_uni, uni))
  return not_uni
run_ttest()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.04452846499912169

In [13]:
def run_ttest():
  data = convert_housing_data_to_quarters().copy()
  data = data.loc[:,'2008q3':'2009q2']
  data = data.reset_index()
  def price_ratio(row):
    return (row['2008q3'] - row['2009q2'])/row['2008q3']

  data['up&down'] = data.apply(price_ratio,axis=1) 
  uni_town = get_list_of_university_towns()['RegionName']
  uni_town = set(uni_town)
  def check_uni_town(row):
    if(row['RegionName'] in uni_town):
      return 1
    else:
      return 0
  data['is_uni']=data.apply(check_uni_town,axis=1)
  
  not_uni=data[data['is_uni']==0].loc[:,'up&down'].dropna()
  uni=data[data['is_uni']==1].loc[:,'up&down'].dropna()
  p_val = list(ttest_ind(not_uni, uni))
  return uni
run_ttest()

5        0.229808
6        0.082810
7        0.063001
9        0.089832
11       0.033328
13       0.021687
14       0.091995
17       0.063724
22       0.027214
23       0.087308
24       0.064859
30       0.143470
36       0.097483
39       0.014363
40       0.066479
47       0.134910
50       0.070686
53       0.273528
60       0.012938
61       0.087178
62       0.052837
66       0.014223
67       0.188279
70      -0.120328
72       0.028436
73       0.144552
78       0.049775
79       0.011982
83       0.040663
85       0.015261
           ...   
9898     0.010546
9902    -0.019666
10057    0.000767
10080    0.113401
10087   -0.001877
10133   -0.000294
10136   -0.061380
10138    0.001273
10141    0.102823
10207    0.030334
10221   -0.056233
10256    0.002440
10263   -0.027301
10284    0.042998
10406    0.164143
10409   -0.081586
10427    0.134069
10440    0.077554
10477    0.057931
10522    0.054549
10529    0.071916
10536   -0.040212
10582   -0.039574
10606    0.058174
10635    0

In [22]:
def run_ttest():
  data = convert_housing_data_to_quarters().copy()
  data = data.loc[:,'2008q3':'2009q2']
  data = data.reset_index()
  def price_ratio(row):
    return (row['2008q3'] - row['2009q2'])/row['2008q3']

  data['up&down'] = data.apply(price_ratio,axis=1) 
  uni_town = get_list_of_university_towns()['RegionName']
  uni_town = set(uni_town)
  def check_uni_town(row):
    if(row['RegionName'] in uni_town):
      return 1
    else:
      return 0
  data['is_uni']=data.apply(check_uni_town,axis=1)
  
  not_uni=data[data['is_uni']==0].loc[:,'up&down'].dropna()
  uni=data[data['is_uni']==1].loc[:,'up&down'].dropna()
  def check_mean():
    if(not_uni.mean()<uni.mean()):
      return 'non-university town'
    else:
      return 'university town'
  p_val = ttest_ind(not_uni, uni)[1]
  return (p_val<0.01,p_val,check_mean())
run_ttest()

(True, 0.0003664160159552697, 'university town')

In [0]:

    uni_town

{'Abilene',
 'Ada',
 'Adrian',
 'Alamosa',
 'Albany',
 'Albion',
 'Alfred',
 'Allendale',
 'Alliance',
 'Alma',
 'Alpine',
 'Altoona',
 'Alva',
 'Ames',
 'Amherst',
 'Angwin',
 'Ann Arbor',
 'Annapolis',
 'Annville',
 'Appleton',
 'Arcata',
 'Arkadelphia',
 'Ashland',
 'Athens',
 'Atlanta',
 'Auburn',
 'Augusta',
 'Aurora',
 'Austin',
 'Ave Maria',
 'Baldwin City',
 'Banner Elk',
 'Bar Harbor',
 'Baton Rouge',
 'Beaumont',
 'Bellingham',
 'Bemidji',
 'Berea',
 'Berkeley',
 'Berrien Springs',
 'Bethlehem',
 'Big Rapids',
 'Binghamton',
 'Blacksburg',
 'Bloomington',
 'Bloomsburg',
 'Bluffton',
 'Boca Raton',
 'Boiling Springs',
 'Bolivar',
 'Boone',
 'Boston',
 'Boulder',
 'Bowling Green',
 'Bozeman',
 'Bradford',
 'Bridgewater',
 'Brockport',
 'Brookings',
 'Brunswick',
 'Buckhannon',
 'Buffalo',
 'Buies Creek',
 'Burlington',
 'California',
 'Cambridge',
 'Canton',
 'Canyon',
 'Cape Girardeau',
 'Carbondale',
 'Carlisle',
 'Carrollton',
 'Castleton',
 'Cecil B. Moore, Philadelphia, al